In [1]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()

#TASK 1 - DO EDA and understand a little about the data.
#Only important thing is to know that it has a lot of features that don't make sense, just a
#bunch of readings from sensors.
#We think many of these features are redundant or irrelevant, and we want to find good features.


<ipython-input-1-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-1-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892


from matplotlib import pyplot as plt
_df_0[0].plot(kind='hist', bins=20, title=0)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1[1].plot(kind='hist', bins=20, title=1)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2[2].plot(kind='hist', bins=20, title=2)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3[3].plot(kind='hist', bins=20, title=3)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series[40]
  ys = series[3]
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_11.sort_values(40, ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel(40)
_ = plt.ylabel(3)

from matplotlib import pyplot as plt
_df_12[0].plot(kind='line', figsize=(8, 4), title=0)
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13[1].plot(kind='line', figsize=(8, 4), title=1)
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14[2].plot(kind='line', figsize=(8, 4), title=2)
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_15[3].plot(kind='line', figsize=(8, 4), title=3)
plt.gca().spines[['top', 'right']].set_visible(False)

In [4]:
for column in df.columns:
  if df[column].dtype == 'object':
    print(f"Column '{column} is likely categorical")
print(f"There is no categorical values")

There is no categorical values


In [ ]:
df.isna().sum()

,0
0,0
1,0
2,0
3,0
4,0
...,...
556,0
557,0
558,0
559,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, 0 to 560
dtypes: float64(561)
memory usage: 31.5 MB


In [ ]:
df.shape

(7352, 561)

In [ ]:
df[1].unique()

array([-0.02029417, -0.01641057, -0.01946716, ..., -0.01701062,
       -0.01884304, -0.01242312])

In [ ]:
df[3].unique()

array([-0.9952786 , -0.99824528, -0.99537956, ..., -0.21821818,
       -0.21913944, -0.26927044])

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xtrain_scaled = scaler.fit_transform(df)

In [6]:
#SCALING
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(df,y,test_size=0.2)


In [ ]:
xtrain_scaled

array([[ 1.65183567,  0.14553573,  0.74138431, ..., -0.59585711,
        -0.14877594,  1.01107792],
       [-0.01973999, -0.05528298,  0.03403627, ...,  1.58794914,
        -0.36781127,  0.87068687],
       [ 0.48820169,  0.04612992,  0.48570301, ..., -0.18489434,
         2.22057179, -3.04441348],
       ...,
       [-4.2391191 , -3.44740641,  3.36839837, ..., -0.06910924,
         2.12730315, -1.06073418],
       [-0.01592274, -0.09966931, -0.13899311, ...,  1.46457201,
        -0.36303414,  0.87633444],
       [-1.55438189, -0.99401062, -1.87547978, ..., -1.25625012,
        -0.11255551,  0.45266886]])

In [7]:
#TASK 5 - 1. Create a pipeline using Gaussian Naive Bayes
#         2. Fit the model to the training data
#         3. Predict values for test set
#         4. Print accuracy score

!pip install scikit-learn

In [9]:
# SOLUTION FOR TASK 5 and 6

# Define a Pipeline with Gaussian Naive Bayes and train it on the full dataset
from sklearn.naive_bayes import GaussianNB
start_time = time.time()
classifier_pipeline_full = Pipeline([
    ('classifier', GaussianNB())
])
classifier_pipeline_full.fit(xtrain, ytrain)
y_pred_full = classifier_pipeline_full.predict(xtest)
end_time = time.time()
full_features_time = end_time - start_time
accuracy_full = accuracy_score(ytest, y_pred_full)

print(f"The accuracy is: {accuracy_full}")
print(f"The time taken is : {full_features_time}")

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The accuracy is: 0.7253569000679809
The time taken is : 0.3496699333190918


In [10]:
import numpy as np
from sklearn.cluster import KMeans

n_clusters = 50
kmeans = KMeans(n_clusters = n_clusters, random_state = 42)

kmeans.fit (xtrain.T)

closest_features = []
for i in range(n_clusters):
  cluster_center = kmeans.cluster_centers_[i]
  distances = np.linalg.norm(xtrain.T -cluster_center, axis =1 )
  closest_feature_index = np.argmin(distances)
  closest_features.append(closest_feature_index)

selected_features_indices = closest_features
selected_features = xtrain.columns[selected_features_indices]

In [11]:
selected_features.shape

(50,)

#TASK 8 - Train another model (GaussianNB) on the new dataset, and report time taken and accuracy


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
import time

x = df.iloc[:,:-1]
y=pd.DataFrame(y)

if pd.api.types.is_numeric_dtype(y.iloc[:,0].dtype):
  encoder = LabelEncoder()
  y = encoder.fit_transform(y)
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=42)

pipeline = Pipeline([
     ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with mean
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB()),
])

start_time = time.time()

pipeline.fit(xtrain,ytrain)

end_time = time.time()

ypred = pipeline.predict(xtest)

accuracy = accuracy_score(ytest,ypred)
print(f"The Accuracy is : {accuracy}")
print(f"The time taken is: {end_time - start_time} seconds")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The Accuracy is : 0.7239972807613868
The time taken is: 0.1703486442565918 seconds
